In [177]:
#from pathlib import Path
import os
from dotenv import load_dotenv
import requests as rq
import certifi
#import urllib3
#urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning )
load_dotenv()


True

In [178]:
CWA_API_KEY=os.getenv("CWA_API_KEY")
print(CWA_API_KEY)

CWA-D580DCE8-4C03-4451-8323-529D0E5B77B3


In [179]:
import os
#CERT_FILE = os.environ.get("SSL_CERT_FILE")
#print(CERT_FILE)

In [180]:
# #跳過ssl憑證驗證verify=False
# location_name = "新北市"
# url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-091"
# params = {
#     "Authorization": CWA_API_KEY,
#     "format": "JSON",
#     "LocationName":location_name
# }
# response = rq.get(url, params=params, verify=False)
# print(response.json())

In [181]:
location_name = "新北市"
base_url = os.getenv("CWA_API_BASE")
endpoint = os.getenv("WEATHER_ENDPOINT")
#print(endpoint)
url = f"{base_url}/{endpoint}"
#print(url)
params = {
    "Authorization": CWA_API_KEY,
    "format": "JSON",
    "LocationName":location_name
}
response = rq.get(url, params=params, verify=certifi.where())
data = response.json()
print(type(data))


<class 'dict'>


In [182]:
response.raise_for_status()

In [183]:
weather_elements = data['records']['Locations'][0]['Location'][0]['WeatherElement']
#print(weather_elements)
"""
print(type(weather_elements))
if type(weather_elements) == list:
    print(len(weather_elements))
"""
temp_el = next((item for item in weather_elements if item['ElementName'] == '平均溫度'), None)

pop_el = next((item for item in weather_elements if item['ElementName'] == '12小時降雨機率'), None)
wx_el = next((item for item in weather_elements if item['ElementName'] == '天氣現象'), None)
if not all([temp_el, pop_el, wx_el]):
     print('天氣資料格式不完整')
hourly_temp, hourly_pop, hourly_wx = temp_el['Time'], pop_el['Time'], wx_el['Time']
print(temp_el)




{'ElementName': '平均溫度', 'Time': [{'StartTime': '2025-07-21T18:00:00+08:00', 'EndTime': '2025-07-22T06:00:00+08:00', 'ElementValue': [{'Temperature': '26'}]}, {'StartTime': '2025-07-22T06:00:00+08:00', 'EndTime': '2025-07-22T18:00:00+08:00', 'ElementValue': [{'Temperature': '30'}]}, {'StartTime': '2025-07-22T18:00:00+08:00', 'EndTime': '2025-07-23T06:00:00+08:00', 'ElementValue': [{'Temperature': '28'}]}, {'StartTime': '2025-07-23T06:00:00+08:00', 'EndTime': '2025-07-23T18:00:00+08:00', 'ElementValue': [{'Temperature': '30'}]}, {'StartTime': '2025-07-23T18:00:00+08:00', 'EndTime': '2025-07-24T06:00:00+08:00', 'ElementValue': [{'Temperature': '27'}]}, {'StartTime': '2025-07-24T06:00:00+08:00', 'EndTime': '2025-07-24T18:00:00+08:00', 'ElementValue': [{'Temperature': '28'}]}, {'StartTime': '2025-07-24T18:00:00+08:00', 'EndTime': '2025-07-25T06:00:00+08:00', 'ElementValue': [{'Temperature': '26'}]}, {'StartTime': '2025-07-25T06:00:00+08:00', 'EndTime': '2025-07-25T18:00:00+08:00', 'ElementV

In [189]:
formatted_weather = []
for temp_time in hourly_temp:
    # print(f"temp_starttime:{temp_time['StartTime']}")
    # #print(reversed(hourly_pop))
    # for p in hourly_pop:
    #      print(f"p_startime:{p['StartTime']}")
    #      if p['StartTime'] <= temp_time['StartTime']:
    #           print ("append to list")
    pop_entry = next((p for p in reversed(hourly_pop) if p['StartTime'] <= temp_time['StartTime']), None)
    #print(pop_entry)
    wx_entry = next((w for w in reversed(hourly_wx) if w['StartTime'] <= temp_time['StartTime']), None)
    formatted_weather.append({
        'time': temp_time['StartTime'], 'temp': temp_time['ElementValue'][0]['Temperature'],
        'pop': pop_entry['ElementValue'][0]['ProbabilityOfPrecipitation'] if pop_entry else 'N/A',
        'wx': wx_entry['ElementValue'][0]['Weather'] if wx_entry else 'N/A'
        #'wxCode': wx_entry['ElementValue'][1]['value'] if wx_entry else 'N/A'
    
    })

print(formatted_weather)

[{'time': '2025-07-21T18:00:00+08:00', 'temp': '26', 'pop': '40', 'wx': '陰時多雲短暫陣雨或雷雨'}, {'time': '2025-07-22T06:00:00+08:00', 'temp': '30', 'pop': '20', 'wx': '陰天'}, {'time': '2025-07-22T18:00:00+08:00', 'temp': '28', 'pop': '20', 'wx': '多雲時陰'}, {'time': '2025-07-23T06:00:00+08:00', 'temp': '30', 'pop': '30', 'wx': '晴午後短暫雷陣雨'}, {'time': '2025-07-23T18:00:00+08:00', 'temp': '27', 'pop': '40', 'wx': '多雲短暫陣雨或雷雨'}, {'time': '2025-07-24T06:00:00+08:00', 'temp': '28', 'pop': '80', 'wx': '多雲時陰短暫陣雨或雷雨'}, {'time': '2025-07-24T18:00:00+08:00', 'temp': '26', 'pop': '90', 'wx': '多雲時陰短暫陣雨或雷雨'}, {'time': '2025-07-25T06:00:00+08:00', 'temp': '27', 'pop': '-', 'wx': '陰短暫陣雨或雷雨'}, {'time': '2025-07-25T18:00:00+08:00', 'temp': '26', 'pop': '-', 'wx': '多雲短暫陣雨或雷雨'}, {'time': '2025-07-26T06:00:00+08:00', 'temp': '30', 'pop': '-', 'wx': '多雲短暫陣雨或雷雨'}, {'time': '2025-07-26T18:00:00+08:00', 'temp': '28', 'pop': '-', 'wx': '多雲短暫陣雨或雷雨'}, {'time': '2025-07-27T06:00:00+08:00', 'temp': '30', 'pop': '-', 'wx': '多雲短暫陣

In [185]:
"""
temp_el = next((item for item in weather_elements if item['elementName'] == 'T'), None)
pop_el = next((item for item in weather_elements if item['elementName'] == 'PoP12h'), None)
wx_el = next((item for item in weather_elements if item['elementName'] == 'Wx'), None)
if not all([temp_el, pop_el, wx_el]):
    return jsonify({'message': '天氣資料格式不完整'}), 500
hourly_temp, hourly_pop, hourly_wx = temp_el['time'], pop_el['time'], wx_el['time']
formatted_weather = []
for temp_time in hourly_temp:
    pop_entry = next((p for p in reversed(hourly_pop) if p['startTime'] <= temp_time['startTime']), None)
    wx_entry = next((w for w in reversed(hourly_wx) if w['startTime'] <= temp_time['startTime']), None)
    formatted_weather.append({
        'time': temp_time['startTime'], 'temp': temp_time['elementValue'][0]['value'],
        'pop': pop_entry['elementValue'][0]['value'] if pop_entry else 'N/A',
        'wx': wx_entry['elementValue'][0]['value'] if wx_entry else 'N/A',
        'wxCode': wx_entry['elementValue'][1]['value'] if wx_entry else 'N/A'
    })
return jsonify(formatted_weather[:24])
"""

"\ntemp_el = next((item for item in weather_elements if item['elementName'] == 'T'), None)\npop_el = next((item for item in weather_elements if item['elementName'] == 'PoP12h'), None)\nwx_el = next((item for item in weather_elements if item['elementName'] == 'Wx'), None)\nif not all([temp_el, pop_el, wx_el]):\n    return jsonify({'message': '天氣資料格式不完整'}), 500\nhourly_temp, hourly_pop, hourly_wx = temp_el['time'], pop_el['time'], wx_el['time']\nformatted_weather = []\nfor temp_time in hourly_temp:\n    pop_entry = next((p for p in reversed(hourly_pop) if p['startTime'] <= temp_time['startTime']), None)\n    wx_entry = next((w for w in reversed(hourly_wx) if w['startTime'] <= temp_time['startTime']), None)\n    formatted_weather.append({\n        'time': temp_time['startTime'], 'temp': temp_time['elementValue'][0]['value'],\n        'pop': pop_entry['elementValue'][0]['value'] if pop_entry else 'N/A',\n        'wx': wx_entry['elementValue'][0]['value'] if wx_entry else 'N/A',\n        'w

In [186]:
formatted_weather = []
for temp_time in hourly_temp:
    print(f"temp_time: {temp_time}")
    

temp_time: {'StartTime': '2025-07-21T18:00:00+08:00', 'EndTime': '2025-07-22T06:00:00+08:00', 'ElementValue': [{'Temperature': '26'}]}
temp_time: {'StartTime': '2025-07-22T06:00:00+08:00', 'EndTime': '2025-07-22T18:00:00+08:00', 'ElementValue': [{'Temperature': '30'}]}
temp_time: {'StartTime': '2025-07-22T18:00:00+08:00', 'EndTime': '2025-07-23T06:00:00+08:00', 'ElementValue': [{'Temperature': '28'}]}
temp_time: {'StartTime': '2025-07-23T06:00:00+08:00', 'EndTime': '2025-07-23T18:00:00+08:00', 'ElementValue': [{'Temperature': '30'}]}
temp_time: {'StartTime': '2025-07-23T18:00:00+08:00', 'EndTime': '2025-07-24T06:00:00+08:00', 'ElementValue': [{'Temperature': '27'}]}
temp_time: {'StartTime': '2025-07-24T06:00:00+08:00', 'EndTime': '2025-07-24T18:00:00+08:00', 'ElementValue': [{'Temperature': '28'}]}
temp_time: {'StartTime': '2025-07-24T18:00:00+08:00', 'EndTime': '2025-07-25T06:00:00+08:00', 'ElementValue': [{'Temperature': '26'}]}
temp_time: {'StartTime': '2025-07-25T06:00:00+08:00', '

In [187]:
formatted_weather = []
for p_time in hourly_pop:
    print(f"p_time: {p_time}")

p_time: {'StartTime': '2025-07-21T18:00:00+08:00', 'EndTime': '2025-07-22T06:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '40'}]}
p_time: {'StartTime': '2025-07-22T06:00:00+08:00', 'EndTime': '2025-07-22T18:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '20'}]}
p_time: {'StartTime': '2025-07-22T18:00:00+08:00', 'EndTime': '2025-07-23T06:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '20'}]}
p_time: {'StartTime': '2025-07-23T06:00:00+08:00', 'EndTime': '2025-07-23T18:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '30'}]}
p_time: {'StartTime': '2025-07-23T18:00:00+08:00', 'EndTime': '2025-07-24T06:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '40'}]}
p_time: {'StartTime': '2025-07-24T06:00:00+08:00', 'EndTime': '2025-07-24T18:00:00+08:00', 'ElementValue': [{'ProbabilityOfPrecipitation': '80'}]}
p_time: {'StartTime': '2025-07-24T18:00:00+08:00', 'EndTime': '2025-07-25T06:00:00+08:00', 'ElementValue': [{'Probabil

In [188]:
formatted_weather = []
for wx_time in hourly_wx:
    print(f"wx_time: {wx_time}")

wx_time: {'StartTime': '2025-07-21T18:00:00+08:00', 'EndTime': '2025-07-22T06:00:00+08:00', 'ElementValue': [{'Weather': '陰時多雲短暫陣雨或雷雨', 'WeatherCode': '17'}]}
wx_time: {'StartTime': '2025-07-22T06:00:00+08:00', 'EndTime': '2025-07-22T18:00:00+08:00', 'ElementValue': [{'Weather': '陰天', 'WeatherCode': '07'}]}
wx_time: {'StartTime': '2025-07-22T18:00:00+08:00', 'EndTime': '2025-07-23T06:00:00+08:00', 'ElementValue': [{'Weather': '多雲時陰', 'WeatherCode': '05'}]}
wx_time: {'StartTime': '2025-07-23T06:00:00+08:00', 'EndTime': '2025-07-23T18:00:00+08:00', 'ElementValue': [{'Weather': '晴午後短暫雷陣雨', 'WeatherCode': '21'}]}
wx_time: {'StartTime': '2025-07-23T18:00:00+08:00', 'EndTime': '2025-07-24T06:00:00+08:00', 'ElementValue': [{'Weather': '多雲短暫陣雨或雷雨', 'WeatherCode': '15'}]}
wx_time: {'StartTime': '2025-07-24T06:00:00+08:00', 'EndTime': '2025-07-24T18:00:00+08:00', 'ElementValue': [{'Weather': '多雲時陰短暫陣雨或雷雨', 'WeatherCode': '16'}]}
wx_time: {'StartTime': '2025-07-24T18:00:00+08:00', 'EndTime': '202